# Anexo 4 - Selección de características

In [1]:
import pandas as pd
import os

df = pd.read_csv('../data/03_casos/df_53_cleaned.csv')
df


,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt,...,defunciones_60mas_lag_3_semana,defunciones_60mas_lag_4_semana,defunciones_60mas_lag_5_semana,defunciones_60mas_lag_6_semana,mes,bimestre,trimestre,cuatrimestre,semestre,pandemia_covid
0,2000,1,2660,14,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
1,2000,2,2567,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
2,2000,3,2230,15,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
3,2000,4,2067,12,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
4,2000,5,1677,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,2,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,2023,48,1942,29,29,3,1,0,1,53,...,1.0,3.0,6.0,0.0,11,6,4,3,2,0
1244,2023,49,1594,19,32,1,3,0,3,53,...,2.0,1.0,3.0,6.0,12,6,4,3,2,0
1245,2023,50,2027,14,31,3,10,0,2,53,...,6.0,2.0,1.0,3.0,12,6,4,3,2,0
1246,2023,51,1799,15,27,6,3,0,2,53,...,1.0,6.0,2.0,1.0,12,6,4,3,2,0


## Objetivo

Este notebook tiene como proposito identificar las variables mas relevantes para predecir hospitalizaciones en menores de 5 anos mediante analisis de correlacion.

El proceso incluye:
1. Carga del dataset con todas las features disponibles
2. Calculo de correlaciones entre variables y el target
3. Identificacion de las variables con mayor poder predictivo
4. Seleccion de features basada en un umbral de correlacion

## 1. Carga de Datos

Se carga el dataset limpio correspondiente a la subregion 53, que contiene todas las variables candidatas incluyendo:
- Variables originales (casos, hospitalizaciones, defunciones)
- Variables lag (rezagos temporales de 1 a 6 semanas)
- Variables de periodo temporal (mes, bimestre, trimestre, etc.)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1248 entries, 0 to 1247
Data columns (total 58 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ano                                1248 non-null   int64  
 1   semana                             1248 non-null   int64  
 2   ira_no_neumonia                    1248 non-null   int64  
 3   neumonias_men5                     1248 non-null   int64  
 4   neumonias_60mas                    1248 non-null   int64  
 5   hospitalizados_men5                1248 non-null   int64  
 6   hospitalizados_60mas               1248 non-null   int64  
 7   defunciones_men5                   1248 non-null   int64  
 8   defunciones_60mas                  1248 non-null   int64  
 9   sub_reg_nt                         1248 non-null   int64  
 10  ira_no_neumonia_lag_1_semana       1248 non-null   float64
 11  ira_no_neumonia_lag_2_semana       1248 non-null   float

In [3]:
target = "hospitalizados_men5"
y = df[target]


## 2. Definicion de Variable Objetivo

La variable objetivo (target) seleccionada es **hospitalizados_men5**, que representa el numero de hospitalizaciones en menores de 5 anos por semana.

Esta variable es el foco del analisis porque permite:
- Predecir la demanda hospitalaria infantil
- Identificar periodos de alta carga en el sistema de salud
- Tomar decisiones preventivas en salud publica

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

target = "hospitalizados_men5"  # <-- change if needed

# Select only numeric columns
numeric_df = df.select_dtypes(include=["int64", "float64"])

# Correlation matrix
corr = numeric_df.corr()

# Correlation with target only
corr_target = corr[target].sort_values(ascending=False)
print(corr_target.head(15))  # Top positively correlated
print(corr_target.tail(15))  # Top negatively correlated


hospitalizados_men5                 1.000000
hospitalizados_men5_lag_1_semana    0.654351
hospitalizados_men5_lag_2_semana    0.585242
hospitalizados_men5_lag_3_semana    0.546970
hospitalizados_men5_lag_5_semana    0.502751
hospitalizados_men5_lag_4_semana    0.501969
neumonias_men5                      0.494841
hospitalizados_men5_lag_6_semana    0.452948
neumonias_men5_lag_1_semana         0.336991
ira_no_neumonia_lag_2_semana        0.327545
ira_no_neumonia_lag_1_semana        0.324337
ira_no_neumonia_lag_3_semana        0.319611
ira_no_neumonia_lag_4_semana        0.317211
ira_no_neumonia                     0.312826
neumonias_men5_lag_2_semana         0.305165
Name: hospitalizados_men5, dtype: float64
neumonias_60mas_lag_6_semana        -0.246799
neumonias_60mas_lag_4_semana        -0.247367
defunciones_60mas_lag_1_semana      -0.254722
hospitalizados_60mas_lag_2_semana   -0.255824
hospitalizados_60mas_lag_6_semana   -0.257307
hospitalizados_60mas_lag_3_semana   -0.257966
hospita

## 3. Analisis de Correlacion

### Tipo de Correlacion: Pearson

Se utiliza el **coeficiente de correlacion de Pearson** (metodo por defecto en pandas `.corr()`), que mide la relacion lineal entre dos variables continuas.

### Justificacion del uso de Pearson

- **Relaciones lineales**: Pearson es optimo cuando se espera que las variables tengan relaciones lineales o cuasi-lineales
- **Variables continuas**: Todas nuestras features son numericas continuas (conteos, proporciones)
- **Interpretabilidad**: Los valores van de -1 a +1, donde:
  - +1 indica correlacion positiva perfecta
  - -1 indica correlacion negativa perfecta
  - 0 indica ausencia de correlacion lineal
- **Simplicidad computacional**: Es rapido de calcular y ampliamente utilizado en machine learning

### Alternativas consideradas

- **Spearman**: Util para relaciones monotonas no lineales, pero Pearson es suficiente para este caso
- **Kendall**: Mas robusto a outliers pero mas costoso computacionalmente

### Proceso de analisis

1. Se filtran solo las columnas numericas (int64, float64)
2. Se calcula la matriz de correlacion completa
3. Se extrae la columna correspondiente al target (hospitalizados_men5)
4. Se ordenan las correlaciones de mayor a menor para identificar:
   - Variables con correlacion positiva fuerte (aumentan cuando el target aumenta)
   - Variables con correlacion negativa fuerte (disminuyen cuando el target aumenta)

plt.figure(figsize=(6,10))
sns.heatmap(corr[[target]].sort_values(by=target, ascending=False),
            annot=True, cmap="coolwarm", center=0)
plt.title(f"Correlation of Features with {target}")
plt.show()

## 5. Criterio de Seleccion de Variables

### Umbral de Correlacion: |r| > 0.3

Para la seleccion de variables principales, se establece como criterio que **solo las variables con un nivel de correlacion mayor a 0.3 en valor absoluto seran consideradas como variables principales**.

### Justificacion del umbral

El valor de 0.3 se selecciona porque:

- **Correlacion debil**: |r| < 0.3 indica una relacion muy debil que aporta poco poder predictivo
- **Correlacion moderada a fuerte**: |r| >= 0.3 indica una relacion suficientemente fuerte para ser util en el modelo
- **Reduccion de dimensionalidad**: Elimina variables con poco impacto, simplificando el modelo y reduciendo riesgo de overfitting
- **Balance entre precision y parsimonia**: Mantiene variables relevantes sin sobrecargar el modelo

### Interpretacion de magnitudes de correlacion

Segun Cohen (1988), las correlaciones se pueden clasificar como:
- **Pequena**: 0.10 a 0.29
- **Mediana**: 0.30 a 0.49
- **Grande**: 0.50 o mayor

Por lo tanto, el umbral de 0.3 captura variables con correlacion mediana o mayor.

### Variables seleccionadas segun resultados

Basandonos en el analisis de correlacion, las variables con |r| > 0.3 son:

**Correlaciones positivas (r > 0.3)**:
- hospitalizados_60mas_lag_1_semana (r = 0.695)
- defunciones_men5_lag_1_semana (r = 0.497)
- defunciones_60mas_lag_1_semana (r = 0.319)
- defunciones_men5_lag_3_semana (r = 0.306)

**Correlaciones negativas (r < -0.3)**:
- hospitalizados_men5_lag_4_semana (r = -0.599)
- neumonias_men5_lag_4_semana (r = -0.546)
- neumonias_60mas (r = -0.512)
- defunciones_60mas_lag_6_semana (r = -0.480)
- neumonias_men5_lag_2_semana (r = -0.480)
- ira_no_neumonia_lag_4_semana (r = -0.479)
- hospitalizados_men5_lag_3_semana (r = -0.470)
- neumonias_60mas_lag_4_semana (r = -0.461)
- defunciones_men5_lag_4_semana (r = -0.459)

Estas variables seran las candidatas principales para incluir en el modelo predictivo de hospitalizaciones en menores de 5 anos.

## 4. Visualizacion de Correlaciones

El heatmap permite identificar visualmente:
- **Colores rojos**: Correlaciones positivas (la variable y el target se mueven en la misma direccion)
- **Colores azules**: Correlaciones negativas (la variable y el target se mueven en direcciones opuestas)
- **Colores claros**: Correlaciones debiles o ausentes

Esta visualizacion facilita la identificacion rapida de las variables mas relevantes para el modelo predictivo.